In [ ]:
%load_ext autoreload
%autoreload 2
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
from pandas import Timedelta
# from optiml.utils import sf
import time
from optiml.utils.sf import logger, sql_to_df, run_sql, conn, session
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

import plotly
plotly.offline.init_notebook_mode()

In [ ]:
df = sql_to_df("select * from sandbox.stg_wh_params")

print(df.iloc[0][0])

In [ ]:
pre_hooks = [
            """
            CREATE OR REPLACE PROCEDURE json_results(SQL VARCHAR)
            RETURNS VARIANT NOT NULL
            LANGUAGE JAVASCRIPT
            EXECUTE AS CALLER

            AS
            $$
            // This variable will hold a JSON data structure that holds ONE row.
            var row_as_json = {};
            // This array will contain all the rows.
            var array_of_rows = [];
            // This variable will hold a JSON data structure that we can return as
            // a VARIANT.
            // This will contain ALL the rows in a single "value".
            var table_as_json = {};

            // Run SQL statement(s) and get a resultSet.
            var command = SQL;
            var cmd1_dict = {sqlText: command};
            var stmt = snowflake.createStatement(cmd1_dict);

            try {
                var rs = stmt.execute();
            }
            catch (err)  {
                return { "error": err}
            }
            // Read each row and add it to the array we will return.
            var row_num = 1;
            while (rs.next())  {
                // Put each row in a variable of type JSON.
                row_as_json = {};
                // For each column in the row...
                for (var col_num = 0; col_num < rs.getColumnCount(); col_num = col_num + 1) {
                //var col_name = COL_NAMES[col_num];
                var col_name = rs.getColumnName(col_num + 1);
                
                row_as_json[col_name] = rs.getColumnValue(col_num + 1);
                }
                // Add the row to the array of rows.
                array_of_rows.push(row_as_json);
                ++row_num;

                }
            // Put the array in a JSON variable (so it looks like a VARIANT to
            // Snowflake).  The key is "key1", and the value is the array that has
            // the rows we want.
            table_as_json = { "rows" : array_of_rows };

            // Return the rows to Snowflake, which expects a JSON-compatible VARIANT.
            return table_as_json;
            $$
            ;
            """,
            """
            create or replace procedure batch_sql_results(SQLCOMMAND String)
                returns variant not null
                language JavaScript
                EXECUTE AS CALLER
            as
            $$
            /**
            * Stored procedure to execute multiple SQL statements generated from a SQL query
            */
                cmd1_dict = {sqlText: SQLCOMMAND};
                stmt = snowflake.createStatement(cmd1_dict);
                rs = stmt.execute();
                var results = [];
                while (rs.next())  {
                    cmd2_dict = {sqlText: rs.getColumnValue("SQL_COMMAND")};
                    cmd = rs.getColumnValue("SQL_COMMAND");
                    //stmtEx = snowflake.createStatement(cmd2_dict);
                    var stmt = snowflake.createStatement({
                            sqlText: \'CALL json_results(:1)\',
                            binds: [cmd]
                    });
                    rs2 = stmt.execute();
                    rs2.next()
                    var s = rs2.getColumnValue(1);

                    table_as_json = {"sql": cmd, "results" : s };
                    results.push(table_as_json);

                    
                }

                return results;  
            $$
            """,
            """call batch_sql_results('select * from wh_param_ddl')"""
    ]

model_sql = """
create or replace table stg_warehouse_params_test as
select * 
from TABLE(RESULT_SCAN(LAST_QUERY_ID()))
"""

In [ ]:
all_sql = pre_hooks
all_sql.append(model_sql)
all_sql

In [ ]:
run_sql_sproc_sql = """
CREATE OR REPLACE PROCEDURE run_sql_commands(SQLCOMMANDS ARRAY)
                    RETURNS STRING
                    LANGUAGE JAVASCRIPT
                    EXECUTE AS CALLER
                        AS
                        $$
                            var results = [];
                            for (var i = 0; i < SQLCOMMANDS.length; i++) {
                                var cmd = SQLCOMMANDS[i];
                                var stmt = snowflake.createStatement({sqlText: cmd});
                                var rs = stmt.execute();
                                results.push(cmd + " executed successfully");
                            }
                            return results.join("");
                        $$;
"""

In [ ]:
run_sql(run_sql_sproc_sql)

In [ ]:
def escape(sql):
    return sql.replace("'", "\\'")

joined_sql_statements = '\n,\n'.join(f"'{escape(item)}'" for item in all_sql)

run_all_sql = f"""
CALL run_sql_commands(ARRAY_CONSTRUCT({joined_sql_statements}))
"""
print(run_all_sql)

In [ ]:
# a = """call batch_sql_results('select * from wh_param_ddl')"""
# print(a.replace("'","\\'"))

In [ ]:
# run_sql(run_all_sql)

In [ ]:
def wrap_task(sql):
    return f"""
    CREATE OR REPLACE TASK test_task 
    WAREHOUSE = DEMO
    AS\n{sql};
    """

display(sql_to_df(wrap_task(run_all_sql)))
display(sql_to_df("execute task test_task"))

In [ ]:
sql_to_df("show tasks")

In [ ]:
run_sql("drop table sandbox.stg_warehouse_params_test")

In [ ]:
run_sql("drop table sandbox.stg_warehouse_params_test")

In [ ]:
df = sql_to_df("select * from sandbox.stg_warehouse_params_test")

print(df.iloc[0][0])